In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim

# 1 Prepare dataset
batch_size = 64
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
)

train_dataset = datasets.MNIST(
    root="./dataset", train=True, download=True, transform=transform
)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
test_dataset = datasets.MNIST(
    root="./dataset", train=False, download=True, transform=transform
)
test_loader = DataLoader(test_dataset, shuffle=True, batch_size=batch_size)

In [2]:
# 2 MODEL DESIGN
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.l1 = torch.nn.Linear(784, 512)
        self.l2 = torch.nn.Linear(512, 256)
        self.l3 = torch.nn.Linear(256, 128)
        self.l4 = torch.nn.Linear(128, 64)
        self.l5 = torch.nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x)


model = Net()
# 放到GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
# 3. Construct Loss and Optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [3]:
# 4 Train and Test
def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader, 0):
        inputs, target = data
        inputs, target = inputs.to(device), target.to(device)
        optimizer.zero_grad()
        # forward + backward + update
        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print(
                "%d, %5d] loss: %.3f" % (epoch + 1, batch_idx + 1, running_loss / 300)
            )
            running_loss == 0.0

In [4]:
def Test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            inputs, target = data
            inputs, target = inputs.to(device), target.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, dim=1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    print("Accuracy on test set : %d %%" % (100 * correct / total))

In [5]:
if __name__ == "__main__":
    for epoch in range(10):
        train(epoch)
        if epoch % 10 == 9:
            Test()

1,   300] loss: 2.215
1,   600] loss: 3.075
1,   900] loss: 3.489
2,   300] loss: 0.332
2,   600] loss: 0.597
2,   900] loss: 0.825
3,   300] loss: 0.190
3,   600] loss: 0.357
3,   900] loss: 0.511
4,   300] loss: 0.134
4,   600] loss: 0.254
4,   900] loss: 0.367
5,   300] loss: 0.104
5,   600] loss: 0.192
5,   900] loss: 0.282
6,   300] loss: 0.077
6,   600] loss: 0.148
6,   900] loss: 0.224
7,   300] loss: 0.061
7,   600] loss: 0.120
7,   900] loss: 0.182
8,   300] loss: 0.049
8,   600] loss: 0.100
8,   900] loss: 0.149
9,   300] loss: 0.036
9,   600] loss: 0.085
9,   900] loss: 0.121
10,   300] loss: 0.032
10,   600] loss: 0.064
10,   900] loss: 0.097
Accuracy on test set : 97 %
